# 🛠️ Ejemplo Práctico: Aplicación Java con Operaciones CRUD sobre MySQL


## 1. Introducción

En este cuadernillo desarrollaremos una aplicación Java paso a paso que permite realizar operaciones CRUD sobre la tabla `alumnos` de una base de datos MySQL. Este ejemplo pone en práctica todos los conceptos vistos en cuadernillos anteriores (JDBC, PreparedStatement, manejo de conexiones, etc.).



## 2. Estructura del Proyecto

Organizaremos la aplicación en cuatro clases:

- `ConexionBD`: se encarga de la conexión a la base de datos.
- `Alumno`: representa la entidad `alumnos`.
- `AlumnoDAO`: contiene los métodos CRUD.
- `Main`: interfaz por consola para el usuario.

Esta separación mejora la legibilidad, mantenimiento y escalabilidad del proyecto.



## 3. Clase `ConexionBD`

```java
import java.sql.*;

public class ConexionBD {
    private static final String URL = "jdbc:mysql://192.168.7.41:3306/escuela";
    private static final String USUARIO = "SuPeRuSeR";
    private static final String CLAVE = "SuPeR2024Us3R";

    public static Connection conectar() throws SQLException {
        return DriverManager.getConnection(URL, USUARIO, CLAVE);
    }
}
```

💡 Recomendación: Probar esta clase antes de continuar para verificar la conexión.



## 4. Clase `Alumno`

```java
public class Alumno {
    private int id;
    private String nombre;
    private int edad;
    private String curso;

    public Alumno(int id, String nombre, int edad, String curso) {
        this.id = id;
        this.nombre = nombre;
        this.edad = edad;
        this.curso = curso;
    }

    // Getters y Setters
    public int getId() { return id; }
    public String getNombre() { return nombre; }
    public int getEdad() { return edad; }
    public String getCurso() { return curso; }

    public void setId(int id) { this.id = id; }
    public void setNombre(String nombre) { this.nombre = nombre; }
    public void setEdad(int edad) { this.edad = edad; }
    public void setCurso(String curso) { this.curso = curso; }

    @Override
    public String toString() {
        return "ID: " + id + ", Nombre: " + nombre + ", Edad: " + edad + ", Curso: " + curso;
    }
}
```



## 5. Clase `AlumnoDAO` (CRUD)

```java
import java.sql.*;
import java.util.*;

public class AlumnoDAO {

    public void insertar(Alumno alumno) throws SQLException {
        String sql = "INSERT INTO alumnos (id, nombre, edad, curso) VALUES (?, ?, ?, ?)";
        try (Connection con = ConexionBD.conectar();
             PreparedStatement ps = con.prepareStatement(sql)) {
            ps.setInt(1, alumno.getId());
            ps.setString(2, alumno.getNombre());
            ps.setInt(3, alumno.getEdad());
            ps.setString(4, alumno.getCurso());
            ps.executeUpdate();
        }
    }

    public List<Alumno> obtenerTodos() throws SQLException {
        List<Alumno> lista = new ArrayList<>();
        String sql = "SELECT * FROM alumnos";
        try (Connection con = ConexionBD.conectar();
             PreparedStatement ps = con.prepareStatement(sql);
             ResultSet rs = ps.executeQuery()) {
            while (rs.next()) {
                Alumno a = new Alumno(
                    rs.getInt("id"),
                    rs.getString("nombre"),
                    rs.getInt("edad"),
                    rs.getString("curso")
                );
                lista.add(a);
            }
        }
        return lista;
    }

    public void actualizar(Alumno alumno) throws SQLException {
        String sql = "UPDATE alumnos SET nombre = ?, edad = ?, curso = ? WHERE id = ?";
        try (Connection con = ConexionBD.conectar();
             PreparedStatement ps = con.prepareStatement(sql)) {
            ps.setString(1, alumno.getNombre());
            ps.setInt(2, alumno.getEdad());
            ps.setString(3, alumno.getCurso());
            ps.setInt(4, alumno.getId());
            ps.executeUpdate();
        }
    }

    public void eliminar(int id) throws SQLException {
        String sql = "DELETE FROM alumnos WHERE id = ?";
        try (Connection con = ConexionBD.conectar();
             PreparedStatement ps = con.prepareStatement(sql)) {
            ps.setInt(1, id);
            ps.executeUpdate();
        }
    }
}
```



## 6. Clase `Main` (Menú Interactivo)

```java
import java.util.*;

public class Main {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        AlumnoDAO dao = new AlumnoDAO();

        while (true) {
            System.out.println("\nMENÚ DE OPCIONES CRUD");
            System.out.println("1. Insertar nuevo alumno");
            System.out.println("2. Listar todos los alumnos");
            System.out.println("3. Actualizar alumno existente");
            System.out.println("4. Eliminar alumno");
            System.out.println("5. Salir");
            System.out.print("Selecciona una opción: ");
            int opcion = sc.nextInt(); sc.nextLine();

            try {
                switch (opcion) {
                    case 1:
                        System.out.print("ID: "); int id = sc.nextInt(); sc.nextLine();
                        System.out.print("Nombre: "); String nombre = sc.nextLine();
                        System.out.print("Edad: "); int edad = sc.nextInt(); sc.nextLine();
                        System.out.print("Curso: "); String curso = sc.nextLine();
                        dao.insertar(new Alumno(id, nombre, edad, curso));
                        System.out.println("Alumno insertado correctamente.");
                        break;
                    case 2:
                        List<Alumno> alumnos = dao.obtenerTodos();
                        if (alumnos.isEmpty())
                            System.out.println("No hay alumnos registrados.");
                        else
                            alumnos.forEach(System.out::println);
                        break;
                    case 3:
                        System.out.print("ID del alumno a actualizar: "); int idUpd = sc.nextInt(); sc.nextLine();
                        System.out.print("Nuevo nombre: "); String nomUpd = sc.nextLine();
                        System.out.print("Nueva edad: "); int edadUpd = sc.nextInt(); sc.nextLine();
                        System.out.print("Nuevo curso: "); String cursoUpd = sc.nextLine();
                        dao.actualizar(new Alumno(idUpd, nomUpd, edadUpd, cursoUpd));
                        System.out.println("Alumno actualizado correctamente.");
                        break;
                    case 4:
                        System.out.print("ID a eliminar: "); int idDel = sc.nextInt();
                        dao.eliminar(idDel);
                        System.out.println("Alumno eliminado correctamente.");
                        break;
                    case 5:
                        System.out.println("Gracias por utilizar el sistema. ¡Hasta pronto!");
                        return;
                    default:
                        System.out.println("Opción no válida. Inténtalo de nuevo.");
                }
            } catch (Exception e) {
                System.out.println("Se ha producido un error: " + e.getMessage());
            }
        }
    }
}
```
